In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/Users/sebas/projects/metabolomics')
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [ ]:
from src.transformers.sklearn_model import SklearnModel
import gensim
from src.load_data import LoadData
from src.config import Config
from tqdm import tqdm
from src.loader_saver import LoaderSaver
import itertools
import numpy as np
from scipy.stats import spearmanr
import dill
from src.plotting import Plotting
from src.load_data import LoadData
from src.molecule_pairs_opt import MoleculePairsOpt
from sklearn.model_selection import train_test_split
from src.train_utils import TrainUtils
from src.preprocessor import Preprocessor
import pickle
import sys
from src.config import Config
from src.parser import Parser
from datetime import datetime
from src.loader_saver import LoaderSaver
from src.molecular_pairs_set import MolecularPairsSet
from scipy.stats import spearmanr
from src.transformers.embedder import Embedder
from src.transformers.encoder import Encoder
import matplotlib.pyplot as plt
from src.transformers.CustomDatasetEncoder import CustomDatasetEncoder
from src.transformers.load_data_encoder import LoadDataEncoder
from torch.utils.data import DataLoader
from src.analog_discovery.cosine_similarity import CosineSimilarity
from rdkit import Chem
from matchms.importing import load_from_mgf,load_from_msp
from matchms.similarity import ModifiedCosine
from src.spec2vec_comparison import Spec2VecComparison
from spec2vec import Spec2Vec
import tensorflow as tf
from src.load_data import LoadData

## params

In [ ]:
ls /Users/sebas/projects/data/

In [ ]:
data_folder= '/Users/sebas/projects/data/'
gnps_path =  data_folder + 'ALL_GNPS_NO_PROPOGATED_wb.mgf'
janssen_path = data_folder + 'drug_plus.mgf'
nist_path = data_folder + 'hr_msms_nist_all.MSP'
output_janssen_file= data_folder + 'all_spectrums_janssen.pkl'
dataset_path= data_folder +'merged_gnps_nist_20240311_unique_smiles_1_million.pkl'
model_path = data_folder + 'best_model_20240319_v2_512u_5_layers.ckpt'
model_spec2vec_file = data_folder + 'spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model'
model_ms2d_file = data_folder + 'ms2deepscore_positive_10k_1000_1000_1000_500.hdf5'
casmi_file= '/Users/sebas/projects/data/processed_massformer/spec_df.pkl' 

In [ ]:
config=Config()

In [ ]:
config.D_MODEL=512
config.N_LAYERS=5

In [ ]:
similarity_name= 'spec2vec'
# use casmi or janssen
USE_CASMI=True

## open janssen data

In [ ]:
all_spectrums_janssen_matchms = [s for s in load_from_mgf(janssen_path)]

In [ ]:
if USE_CASMI:
    all_spectrums_janssen_su=LoadData.get_all_spectrums_casmi(
        casmi_file,
        config=config,
    )
else:
    loader_saver = LoaderSaver(
            block_size=100,
            pickle_nist_path='',
            pickle_gnps_path='',
            pickle_janssen_path=output_janssen_file,
        )
    all_spectrums_janssen_su = loader_saver.get_all_spectrums(
                janssen_path,
                100000000,
                use_tqdm=True,
                use_nist=False,
                config=config,
                use_janssen=True,
            )

In [ ]:
pp=Preprocessor()
### remove extra peaks in janssen
all_spectrums_janssen_processed = [pp.preprocess_spectrum(
            s,
            fragment_tol_mass=10,
            fragment_tol_mode="ppm",
            min_intensity=0.01,
            max_num_peaks=100,
            scale_intensity=None,
        ) for s in all_spectrums_janssen_processed]

all_spectrums_janssen = [s_original for s_original, s_processed in zip(all_spectrums_janssen_original,all_spectrums_janssen_processed) if len(s_processed.mz)>=6]

## open ref data

In [ ]:
with open(dataset_path, 'rb') as file:
            dataset = dill.load(file)

In [ ]:
all_spectrums_reference_su= dataset['molecule_pairs_train'].spectrums_original +\
                    dataset['molecule_pairs_val'].spectrums_original + \
                        dataset['molecule_pairs_test'].spectrums_original

In [ ]:
all_spectrums_gnps_matchms = [s for s in load_from_mgf(gnps_path)]

In [ ]:
all_spectrums_nist_matchms=  [s for s in load_from_msp(nist_path)]

In [ ]:
all_spectrums_reference_matchms = all_spectrums_gnps_matchms + all_spectrums_nist_matchms

In [ ]:
#compute all the hashes from the datasets
target_hashes = [s.spectrum_hash for s in all_spectrums_reference_su]

In [ ]:
matchms_hashes=[]
for s in  tqdm(all_spectrums_reference_matchms):
    matchms_hashes.append(s.spectrum_hash())

In [ ]:
len(target_hashes)

In [ ]:
len(all_spectrums_reference_matchms)

In [ ]:
matchms_hashes.index(target_hashes[0])

In [ ]:
indexes_matched = [matchms_hashes.index(t) for t in tqdm(target_hashes)]

In [ ]:
all_spectrums_reference = [all_spectrums_reference_matchms[index] for index in indexes_matched]

In [ ]:
for i,(s_ref, s_su) in enumerate(zip(all_spectrums_reference, all_spectrums_reference_su)):
    new_metadata=s_ref.metadata.copy()
    new_metadata['smiles']=s_su.smiles
    all_spectrums_reference[i].metadata=new_metadata

## Filter spectra from Janssen

In [ ]:
su_hashes = [s.spectrum_hash for s in all_spectrums_janssen_su]

In [ ]:
all_spectrums_janssen = [s for s in all_spectrums_janssen_matchms if s.spectrum_hash() in su_hashes]

## Find those instances that are not in reference

In [ ]:
canon_smiles_reference = [Chem.CanonSmiles(s.metadata['smiles']) for s in all_spectrums_reference]
canon_smiles_janssen =   [Chem.CanonSmiles(s.metadata['smiles']) for s in all_spectrums_janssen]
janssen_indexes_in_ref= [i for i,s in enumerate(canon_smiles_janssen) if s in canon_smiles_reference]
janssen_indexes_not_in_ref = [i for i,s in enumerate(canon_smiles_janssen) if s not in canon_smiles_reference]

In [ ]:
len(janssen_indexes_in_ref),len(janssen_indexes_not_in_ref)

In [ ]:
all_spectrums_janssen = [all_spectrums_janssen[index] for index in janssen_indexes_in_ref]

In [ ]:
filtered_canon_smiles_janssen= [Chem.CanonSmiles(s.metadata['smiles']) for s in all_spectrums_janssen]

In [ ]:
# dictionary indexed by smiles in janssen, and corresponding reference spectra
#dict_spectra_ref_mapped = {s:[all_spectrums_reference[i] for i,t in enumerate(canon_smiles_reference) if t==s ]  for s in (filtered_canon_smiles_janssen)}

## load model

In [ ]:
#encoder= Encoder(model_path, D_MODEL=int(config.D_MODEL),N_LAYERS=int(config.N_LAYERS))

In [ ]:
#similarity_model =ModifiedCosine(tolerance=0.1)

In [ ]:
if similarity_name=='spec2vec':
    model = gensim.models.Word2Vec.load(model_spec2vec_file)
    similarity_model= Spec2Vec(
                model=model, intensity_weighting_power=0.5, allowed_missing_percentage=100.0
            )
    PREPROCESS_SPECTRUMS=True
elif similarity_name=='modified_cosine':
    similarity_model =ModifiedCosine(tolerance=0.1)
    PREPROCESS_SPECTRUMS=True
elif similarity_name == 'ms2deepscore':
    model_ms2d = load_model(model_ms2d_file)
    similarity_model= MS2DeepScore(model_ms2d)

## Preprocessed spectrums

In [ ]:
preprocessed_all_spectrums_janssen =all_spectrums_janssen.copy()
if PREPROCESS_SPECTRUMS:
    for i,s in tqdm(enumerate(preprocessed_all_spectrums_janssen)):
        preprocessed_all_spectrums_janssen[i] = Spec2VecComparison.spectrum_processing(preprocessed_all_spectrums_janssen[i])
preprocessed_all_spectrums_janssen = [s for s in preprocessed_all_spectrums_janssen if ((s is not None)and (s.metadata['precursor_mz']>0))]

In [ ]:
preprocessed_all_spectrums_reference =all_spectrums_reference.copy()
if PREPROCESS_SPECTRUMS:
    for i,s in tqdm(enumerate(preprocessed_all_spectrums_reference)):
        preprocessed_all_spectrums_reference[i] = Spec2VecComparison.spectrum_processing(preprocessed_all_spectrums_reference[i])
preprocessed_all_spectrums_reference = [s for s in preprocessed_all_spectrums_reference if ((s is not None)and (s.metadata['precursor_mz']>0))]

In [ ]:
#with open('preprocessed_all_spectrums_reference.pkl', 'wb') as file:
#        dictionary={'preprocessed_all_spectrums_reference':preprocessed_all_spectrums_reference}
#        dill.dump(dictionary,file)

#with open('preprocessed_all_spectrums_reference.pkl', 'rb') as file:
#        preprocessed_all_spectrums_reference=dill.load(file)['preprocessed_all_spectrums_reference']

## compute similarities

In [ ]:
from matchms import calculate_scores
if similarity_name != 'ms2deepscore':
    results_scores = calculate_scores(
                        preprocessed_all_spectrums_reference, preprocessed_all_spectrums_janssen, similarity_model
                    )
else:
    with tf.device('/device:CPU:0'): #execute on cpu
        results_scores = calculate_scores(
                        preprocessed_all_spectrums_reference, preprocessed_all_spectrums_janssen, similarity_model
                    )

In [ ]:
results_scores

In [ ]:
#results_ordered = [results_scores.scores_by_query(s, name='Spec2Vec', sort=True) for s in preprocessed_all_spectrums_janssen]
#spectrums_retrieved = [r[0] for r in results_ordered]
#max_sim = [r[1] for r in results_ordered]

In [ ]:
## convert from smiles to canon smiles
dict_canon_smiles = {s.metadata['smiles']:Chem.CanonSmiles(s.metadata['smiles']) for s in preprocessed_all_spectrums_reference}

In [ ]:
len(dict_canon_smiles.keys())

In [ ]:
rankings=[]
for spectrum in tqdm(preprocessed_all_spectrums_janssen):
    #print('starting procerssing')
    #print(target_smile)
    target_smile = Chem.CanonSmiles(spectrum.metadata['smiles'])
    
    #results obtained
    #print('getting scores')
    results_ordered = results_scores.scores_by_query(spectrum, name='Spec2Vec', sort=True)
    
    spectrums_retrieved = [r[0] for r in results_ordered]

    #print('getting smiles')
    try: #try only a few smiles
        
        smiles_retrieved = [dict_canon_smiles[r.metadata['smiles']] for r in spectrums_retrieved[0:1000]]
    
        # get the ranking
        rank = smiles_retrieved.index(target_smile)
        #print(rank)
        rankings.append(rank)
    except:
        smiles_retrieved = [dict_canon_smiles[r.metadata['smiles']] for r in spectrums_retrieved]
    
        # get the ranking
        if target_smile in smiles_retrieved:
            rank = smiles_retrieved.index(target_smile)
            rankings.append(rank)

In [ ]:
rankings=np.array(rankings)

In [ ]:
rankings_x = np.arange(0,400)
rankings_proportion = [rankings[rankings<=r].shape[0]/rankings.shape[0] for r in rankings_x]

In [ ]:
results = {'rankings_x':rankings_x,
          'rankings_y': rankings_proportion,
           'rankings':rankings}

In [ ]:
with open('./notebooks/discovery_search/results/'+similarity_name + '_results_analog_discovery_ranking.pkl', 'wb') as f:
    dill.dump(results, f)